In [20]:
#### Import libraries:

import pandas as pd
import numpy as np 

In [2]:
#### Create final dataset of US team skiers' resort details: 

In [3]:
# Load first ski resorts details dataset 
Kaggle = pd.read_csv('path/to/ski_resorts_kaggle.csv')

# Rename 'name' to 'Resort' for consistency in merging
Kaggle.rename(columns={'name': 'Resort'}, inplace=True)  

In [4]:
# Standardize resort names in Kaggle dataset to match names in other datasets
Kaggle.loc[Kaggle['Resort'] == 'Copper Mountain Resort', 'Resort'] = 'Copper Mountain'
Kaggle.loc[Kaggle['Resort'] == 'Killington Resort', 'Resort'] = 'Killington'
Kaggle.loc[Kaggle['Resort'] == 'Sierra at Tahoe', 'Resort'] = 'Sierra-at-Tahoe'
Kaggle.loc[Kaggle['Resort'] == 'Winter Park', 'Resort'] = 'Winter Park Resort'

In [5]:
# Load pre-made dataset 
df = pd.read_csv('resortdata.csv') 

# Rename 'resort' to 'Resort' for consistency
df.rename(columns={'resort': 'Resort'}, inplace=True) 

In [6]:
# Standardize resort names in custom dataset to match names in Kaggle datasets
df.loc[df['Resort'] == 'Copper Mountain Resort', 'Resort'] = 'Copper Mountain'
df.loc[df['Resort'] == 'Killington Resort', 'Resort'] = 'Killington'
df.loc[df['Resort'] == 'Crested Butte Mountain Resort', 'Resort'] = 'Crested Butte'
df.loc[df['Resort'] == 'Vail Mountain, Beaver Creek Resort', 'Resort'] = 'Beaver Creek'
df.loc[df['Resort'] == 'Mammoth Mountain Ski Area', 'Resort'] = 'Mammoth Mountain'
df.loc[df['Resort'] == 'Telluride Ski & Golf Resort', 'Resort'] = 'Telluride'
df.loc[df['Resort'] == 'Park City Mountain Resort', 'Resort'] = 'Park City'
df.loc[df['Resort'] == 'Waterville Valley Resort', 'Resort'] = 'Waterville Valley'
df.loc[df['Resort'] == 'Mt. Bachelor LLC', 'Resort'] = 'Mt. Bachelor'
df.loc[df['Resort'] == 'Okemo Mountain Resort', 'Resort'] = 'Okemo'
df.loc[df['Resort'] == 'Vail Mountain', 'Resort'] = 'Vail'
df.loc[df['Resort'] == 'Squaw Valley / Alpine Meadows', 'Resort'] = 'Palisades Tahoe'

In [10]:
# Load second ski resorts details dataset (encoding to handle special characters in file)
Kaggle1 = pd.read_csv('path/to/ski_resorts_kaggle1.csv', encoding= 'ISO-8859-1') 

# Standardize resort names in the second Kaggle dataset to ensure consistency across datasets
Kaggle1.loc[Kaggle1['Resort'] == 'Bald Mountain-Sun Valley', 'Resort'] = 'Sun Valley Resort'
Kaggle1.loc[Kaggle1['Resort'] == 'Squaw Valley', 'Resort'] = 'Palisades Tahoe'
Kaggle1.loc[Kaggle1['Resort'] == 'Northstar California Resort', 'Resort'] = 'Northstar California'

In [11]:
# Perform an inner merge between custom dataset and first Kaggle dataset on 'Resort'
merged_outer = pd.merge(df, Kaggle, on='Resort', how='inner')

In [12]:
# Remove row with Wisconsin for data consistency and fixing errors 
merged_outer = merged_outer[merged_outer['location_region'] != 'Wisconsin']

# Remove duplicate rows by 'Resort' and keep only first occurrence
merged_outer = merged_outer.drop_duplicates(subset='Resort', keep='first')
merged_outer.reset_index(drop=True, inplace=True)

# Merge refined dataset with second Kaggle dataset
merged_inner1 = pd.merge(merged_outer, Kaggle1, on='Resort', how='left')

# Drop unnecessary columns after merging to focus on relevant data
merged_inner2 = merged_inner1.drop(columns=['url', 'location_coordinate', 'location_region', 'Continent', 
                                            'Country', 'Longitude', 'Latitude', 'ID', 'number_of_matches'])

In [ ]:
# Save current DataFrame to CSV file 
merged_inner2.to_csv('completeresort.csv', index=False)

print("Resort dataset created. Continue to next steps to merge resort data with relevant athlete data.")

In [14]:
# Load pre-made dataset 
allUSskiers = pd.read_csv('all_US_skiers.csv') 

# Standardize Skiclub names for consistency to avoid merging issues
allUSskiers.loc[allUSskiers['Skiclub'] == 'Aspen Valley ski and Snowboard Club', 'Skiclub'] = 'Aspen Valley Ski and Snowboard Club'
allUSskiers.loc[allUSskiers['Skiclub'] == 'Winter Park Competition Center', 'Skiclub'] = 'Winter Park competition center'

In [15]:
# Load previously saved resort data
completeresort = pd.read_csv('completeresort.csv') 

# Merge skier and resort datasets on 'Skiclub' column (outer join to keep all records from both datasets)
merged_inner = pd.merge(allUSskiers, completeresort, on='Skiclub', how = 'outer')

In [16]:
# Define the columns we need to focus on for the analysis
columns_to_select = [
    'Competitorname', 'Skiclub', 'HPpoints', 'HPpos', 'state_data', 'region_data',
    'Resort', 'rank', 'rating', 'elevation_top_m', 'elevation_difference_m',
    'number_of_lifts', 'number_of_slopes', 'annual_snowfall_cm', 'Price', 'Season', 
    'Beginner slopes', 'Intermediate slopes', 'Difficult slopes', 'Nightskiing', 'Summer skiing'
]

# Create a new DataFrame
selected_columns_df = merged_inner[columns_to_select]

In [17]:
# Sort the new DataFrame by 'HPpos' in ascending order to rank competitors by position
sorted_by_hp_pos = selected_columns_df.sort_values(by='HPpos', ascending=True)

sorted_by_hp_pos.reset_index(drop=True, inplace=True) # Reset index

# Remove rows where the 'Resort' column has missing values
sorted_by_hp_pos = sorted_by_hp_pos.dropna(subset=['Resort'])

In [21]:
# List of columns where values will be set to NaN for rows with 'Sun Valley Resort'
columns_to_nullify = [
    'rank', 'rating', 'elevation_top_m', 'elevation_difference_m',
    'number_of_lifts', 'number_of_slopes', 'annual_snowfall_cm', 'Price', 
    'Season', 'Beginner slopes', 'Intermediate slopes', 
    'Difficult slopes', 'Nightskiing', 'Summer skiing'
]

# Set specified columns to NaN in rows where the 'Resort' is 'Sun Valley Resort'
sorted_by_hp_pos.loc[sorted_by_hp_pos['Resort'] == 'Sun Valley Resort', columns_to_nullify] = np.nan

# Define a final list of columns to use for the analysis
columns_to_select = [
    'Resort', 'HPpoints', 'HPpos', 'state_data', 'region_data', 'elevation_top_m',
    'number_of_slopes', 'annual_snowfall_cm', 'Price', 'Beginner slopes', 
    'Intermediate slopes', 'Difficult slopes', 'Nightskiing', 'Summer skiing'
]

# Create a DataFrame with the final set of columns, dropping rows with any missing values
selected_columns_df = merged_inner[columns_to_select].dropna()

In [ ]:
# Save the dataset to a new CSV file
selected_columns_df.to_csv('resort_with_athletes.csv', index=False)

print("Final dataset created with all US halfpipe skiers' resort data with resort details.")